In [ ]:
#!/usr/bin/python3
import serial.tools.list_ports
from datetime import datetime
from PIL import Image, ImageTk
from tkinter import filedialog
import tkinter.font as font
from datetime import date
from serial import Serial
from queue import Queue
from tkinter import ttk
from os import listdir
import tkinter as tk
from tkinter import *
import PIL.Image
import pandas as pd
import numpy as np
from array import *
import PIL.Image
import PIL.ImageTk
import serial
import pickle
import base64
import time
import io
import os
import cv2
import base64
import scipy.ndimage
import pandas as pd
import socket
import threading
import shutil
from threading import Thread
import warnings
warnings.filterwarnings("ignore")

#initializing variables
j=counter=k=cnt=0
Number_of_axle=u3=u4=u5=h11=n1=v=v1=v2=v3=v4=v5=v6=gen_str=0
max_value = None

#creating empty list
img,one_list,Number_of_axles,vehicle_hgt,date1,sort_date=[],[],[],[],[],[]

serialobj = serial.Serial()
serialobj.port = "COM2"
serialobj.baudrate= 115200
serialobj.open()

try:
    ob=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    ob.bind(('192.168.0.108', 1020))
    ob.listen(5)
    clientobject, addr = ob.accept()
except OSError:
    pass 
  
root = Tk()
root.config(bg='gray')
root.geometry('700x500')

label = Label(root, text="Vehicle Class", width=22, height=3, bg = "skyblue", font=('Arial', 10, 'bold')) 
label.grid(row=50,column=8, columnspan=2)

message_font = font.Font(size=15)
mesg = Message(root, text="No vehicle", font= message_font, justify = tk.CENTER)  
mesg.grid(row= 60, column=8, columnspan=2, ipadx=45, ipady=45)
mesg.config(bg='lightgreen')

datacanvas = Canvas(root, width = 500, height = 200, bg = 'white')
datacanvas.grid(row=50,column=0, columnspan=4, rowspan= 50)

vsb = Scrollbar(root, orient = 'vertical', command = datacanvas.yview)
vsb.grid(row=50, column=4,columnspan=1, rowspan = 50, sticky = 'ns')

datacanvas.config(yscrollcommand = vsb.set)
dataframe = Frame(datacanvas, bg='white')
datacanvas.create_window((10,0),window=dataframe, anchor='nw')

canvas = tk.Canvas(root, width=700, height=300, bg="white")
canvas.grid(row=0, column=0,columnspan=10, rowspan= 50)
   
d=[]
def binary_fn(str2,j):
    L1,line1=[],[]
    sliced_str = str2[0:29]
    try:
        if len(sliced_str)==29:
            result = bin(int(sliced_str,16))
            c = result[6:]
            c= list(c)
            d.append(c)
            arr = np.array(d, ndmin=2)
            arr = arr.transpose()
            L1 = arr.tolist()
            for i, line in enumerate (L1):
                line = [int(x) for x in line]
                if sum(line)!=0:
                    line1.append(line)   
            arr1 = np.array(line1, ndmin=2)
            arr1 = arr1.astype('uint8')*255
            arr1 = cv2.rotate(arr1, cv2.ROTATE_180)
            (height, width) = arr1.shape[:2]
            if width<50:
                arr1 = cv2.resize(arr1, (width*10, 150))
            elif width>=50 and width<80:
                arr1 = cv2.resize(arr1, (width*6, 150))
            elif width>=80 and width<100:
                arr1 = cv2.resize(arr1, (width*4, 150))
            elif width>=100 and width<120:
                arr1 = cv2.resize(arr1, (width*3, 150))
            elif width>=120 and width<140:
                arr1 = cv2.resize(arr1, (width*4, 150))
            elif width>=150:
                arr1 = cv2.resize(arr1, (width*3, 150))
            kernel = np.ones((5,5), np.uint8)
            _, th1 = cv2.threshold(arr1, 150,255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            c1 = cv2.morphologyEx(th1, cv2.MORPH_CLOSE, kernel)
            mask_inv = cv2.bitwise_not(c1)
            mask_inv = cv2.flip(mask_inv, 1)
            c = ''.join(c)
            save_image(mask_inv)
            axle_number(c)
        else:
            Label(dataframe, text = "Data Lost issue", font = ('calibri','13'), bg='white').pack()
            root.update()
    except PermissionError:
        pass
def save_image(mask_inv):
    cv2.imwrite('C:/Users/Acer/Desktop/programs/pyserial/exp/image.jpg',mask_inv)
def open_image_gui(j):
    try:
        root.img = img = cv2.imread('C:/Users/Acer/Desktop/programs/pyserial/exp/image.jpg')
        im = PIL.Image.fromarray(img)
        imgtk = PIL.ImageTk.PhotoImage(image=im)
        canvas.image = imgtk # keep a reference!
        canvas.create_image((310,130), anchor='center',image=imgtk)
        canvas.update()
        get_data(j)
    except RuntimeError:
        root.destroy()
def axle_number(c):
    global k, Number_of_axle
    if c.startswith('1'):
        k+=1
        one_list.append(str(k))
    elif c.startswith('0'):
        k=0
    Number_of_axles.append(one_list.count('2'))
    axle = Number_of_axles[-1:][0]
    Number_of_axle = str(axle)
    first_two_axle_height(c,axle)
    
u,u1,u2=[],[],[]
def first_two_axle_height(c,axle):
    global u3,u4,u5,u1,u2,u
    if c.startswith('1'):
        c = list(c)
        sec = [x for x in range(len(c)) if c[x] == '1']
        u.append(sec[-1])
    elif c.startswith('0'):
        if u==0:
            pass
        elif u!=0:
            if u==[]:
                pass
            elif u!=[]:
                u2.append(min(u)+1)
                u1.append(max(u)+ 1)
                for i in range (len(u1)):
                    if i == 0:
                        u3= u1[0]
                        u4 = u2[0]
                    elif i==1:
                        u5 = u1[1]
                    u.clear()
    vehicle_height(c, axle)
h12=[]
def vehicle_height(c,axle):
    global h11
    c = list(c)
    res = [x for x in range(len(c)) if c[x] == '1']
    res = res[-1]
    h12.append(res)
    h12.sort()
    h11 = str(max(h12)+1)
    axle_distance(c,axle)
m=0
n=[]
def axle_distance(c,axle):
    global m
    c = list(c)
    if c[0]=='1':
        if c.count('1')>20:
            m+=1
        elif c.count('1')<10:
            m=0
            n.append(m)
    elif c[0]=='0':
        n.append(m)
        m=0
    ground_clearance(c, axle)
w,w1=[],[]
def ground_clearance(c, axle):
    global v,v1,v2,v3,v6
    c = list(c)
    gnd = [x for x in range(len(c)) if c[x] == '1']
    gnd = gnd[0]
    if gnd!=0:
        w.append(gnd)
    else:
        if w!=[]:
            v1 = w[1:]
            v4=w[2:]
            if v1!=[]:
                if axle == 1:
                    for i in v1:
                        if i<10:
                            v2+=1
                        else:
                            v3+=1
            if v4!=[]:
                v5 = v4[:-2]
                v6 = sum(x>4 and x<10 for x in v5)
    Number_of_windows(c)
windows,win=[],[]
def Number_of_windows(c):
    c = list(c)
    if c[0]=='1':
        windows.append(c.count('1'))
    elif c[0]=='0':
        win.append(windows[-2])
        windows.clear()
def text_data(j, recentpacket):
        td1 = date.today()
        td = td1.strftime("%d-%m-%Y")
        directory_path = 'C:/Users/Acer/Desktop/programs/pyserial/Data_rp'
        directory_name = td
        try:
            path = os.path.join(directory_path,directory_name)
            os.makedirs(path, exist_ok = True)
            new_file_path = os.path.join(path, str(j)+'.txt')
            new_file = open(new_file_path,'a')
            new_file.writelines(str(recentpacket))
            new_file.writelines('\n')
            new_file.close()
        except OSError as error:
            pass
def checkSerialPort():
    global cnt,j
    compare_list=['A00FFFFFFFFFFFFFFFFFFFFFFFFFF;,0\r']
    if serialobj.isOpen() and serialobj.in_waiting:
        recentpacket = serialobj.readline()
        text_data(j, recentpacket)
        recentpacketstring = recentpacket.decode('utf').rstrip('\n')
        str1 = ''+ recentpacketstring
        compare_list.append(str1)
        if str1.startswith("A"):
            try:
                for i in range(len(compare_list)):
                        if compare_list[i] == compare_list[i+1]:
                            Label(dataframe, text = 'No vehicle', font = ('calibri','13'), bg='white').pack()
                            root.update()
                            compare_list.clear()
                        else:
                            Label(dataframe, text = str1, font = ('calibri','13'), bg='white').pack()
                            str2 = ''+ recentpacketstring
                            binary_fn(str2,j)        
            except IndexError:
                pass   
        elif str1.startswith("S"):
            open_image_gui(j)
            j+=1
            Number_of_axle,u3,u4,u5,h11,n1,v2,v3=0,0,0,0,0,0,0,0
            first_two_axle_distance, v6 = 0,0
            d.clear()
            Number_of_axles.clear()
            one_list.clear()
            vehicle_hgt.clear()
            n.clear()
            w.clear()
            u.clear()
            u1.clear()
            u2.clear()
            h12.clear()
            win.clear()
            axle_dist.clear()
        else:
            Label(dataframe, text = 'No Vehicle', font = ('calibri','13'), bg='white').pack()
            root.update() 
            pass
nwlist,ax_ht,both_axle_counts, axle_dist=[],[],[], []
tandum='False'
Number_of_axle,u3,u4,u5,h11,n,v2,v3,v6

def get_data(j):
    global max_value, ad, Number_of_axle,tandum, u3,u4,u5,h11,n,v2,v3,v6,win
    Number_of_axle = str(Number_of_axle)
    try:
        len_n = len(n)
        for i in range (len_n):
            if (n[i] != 0 and n[i] != 1):
                axle_dist.append(i)
        first_two_axle_distance = axle_dist[1]-axle_dist[0]
        first_two_axle_distance = int(first_two_axle_distance)*3
        print('Axle_distance:------->', int(first_two_axle_distance))
    except:
        first_two_axle_distance=0
        print('Axle_distance:------->', int(first_two_axle_distance))
    v6 = int(v6)
    axle_ht1 = int(u4)*3
    axle_ht2 = int(u3)*3
    max_hgt = int(h11)*3
    sec_ax_ht1 = int(u5)*3
    gnd_clr_OFF = int(v2)*3
    gnd_clr_ON = int(v3)*3
    try:
        windows = win[-1]
        windows1 = int(windows)
        print("window", windows1)
    except:
        windows1 = 0
        print("window", windows1)
        
    print('Number of axle:------>', int(Number_of_axle))
    print('Max Height of vehicle:------->', int(max_hgt))
    print('Starting_First_Axle_height of vehicle:------->', axle_ht1)
    print('Ending_First_Axle_height of vehicle:------->', axle_ht2)
    print('Second_Axle_height of vehicle:------->', sec_ax_ht1)
    print('Ground Clearance OFF:------->', gnd_clr_OFF)
    print('Ground Clearance ON:------->', gnd_clr_ON)
    axle_ht=ttl_hgt=sec_ax_ht=windows=v2=v3=0 
    try:
        if Number_of_axle == '0':
            txt='0, "Non Tollable"'
            message_box(txt)
        elif Number_of_axle == '1':
            if 70<axle_ht1<120  and  70<sec_ax_ht1<250 and gnd_clr_ON<13:
                txt = '3, CJV'
                message_box(txt)
            else:
                txt='0, "Non Tollable"'
                message_box(txt)
        elif Number_of_axle == '2':
            if axle_ht1<70 and 5<gnd_clr_ON<40:
                txt = '12, "Tractor"'
                message_box(txt)
            elif axle_ht1<70 and gnd_clr_ON<10:
                txt = '2, "3 Wheeler"'
                message_box(txt)
            elif 70<axle_ht1<120  and  70<sec_ax_ht1<270 and gnd_clr_ON<13:
                txt = '3, "CJV"'
                message_box(txt)
            elif axle_ht1<90:
                txt = '1, "2 Wheeler"'
                message_box(txt)
            elif 100<axle_ht1<240:
                txt = '4, "LCV"'
                message_box(txt)
            elif axle_ht1>240 and windows1< 100:
                txt = '6, "Bus"'
                message_box(txt)
            elif axle_ht1>240 and gnd_clr_ON<gnd_clr_OFF:
                txt = '6, "Bus"'
                message_box(txt)
            else:
                txt = '7, "2 Axle truck"'
                message_box(txt)
        elif Number_of_axle == '3':
            if 70<axle_ht1<110  and  70<sec_ax_ht1<270 and gnd_clr_ON<13:
                txt = '3, "CJV"'
                message_box(txt)
            elif axle_ht1<90:
                txt = '1, "2 Wheeler"'
                message_box(txt)
            elif 80<axle_ht1<150:
                txt = '13, "Tractor With Trolley"'
                message_box(txt)
            elif axle_ht1>240 and windows1< 100:
                txt = '6, "Bus"'
                message_box(txt)
            elif axle_ht1>240 and gnd_clr_ON<gnd_clr_OFF:
                txt = '6, "Bus"'
                message_box(txt)
            else:
                txt = '8, "3 Axle truck"'
                message_box(txt)
        elif Number_of_axle > '3' and Number_of_axle <= '6':
            if 70<axle_ht1<120  and  70<sec_ax_ht1<270 and gnd_clr_ON<13:
                txt = '3, "CJV"'
                message_box(txt)
            elif axle_ht1<90 and gnd_clr_ON<13:
                txt = '1, "2 Wheeler"'
                message_box(txt)
            elif axle_ht1<160:
                txt = '13, "Tractor With Trolley"'
                message_box(txt)
            elif v6>10:
                tandum = 'True'
                txt = '9, "MAV"'
                message_box(txt)
                v6=0
            else:
                txt = '9, "MAV"'
                message_box(txt)
        elif Number_of_axle > '6':
            if 70<axle_ht1<120  and  70<sec_ax_ht1<270 and gnd_clr_ON<13:
                txt = '3, "CJV"'
                message_box(txt)
            elif 70<axle_ht1<120:
                txt = '3, "CJV"'
                message_box(txt)
            else:
                txt = '10, "OSV"'
                message_box(txt)
    except UnboundLocalError:
        txt='0, "Non Tollable"'
        message_box(txt)
    image_to_base64(j, txt, Number_of_axle, max_hgt, first_two_axle_distance, tandum)
def message_box(txt):
    message_font = font.Font(size=13)
    mesg = Message(root, text= txt, font= message_font, justify = tk.CENTER)  
    mesg.grid(row= 60, column=8, columnspan=2, ipadx=30, ipady=45)
    mesg.config(bg='lightgreen')
    print(txt)
    txt = 'no vehicle'        
def image_to_base64(j, txt, Number_of_axle, max_hgt, first_two_axle_distance, tandum):
    global gen_str
    with open("C:/Users/Acer/Desktop/programs/pyserial/exp/image.jpg", 'rb') as fs:
        my_string = base64.b64encode(fs.read())
        my_string1 = my_string.decode('utf-8')
        td1 = date.today()
        td = td1.strftime("%d-%m-%Y")
        t = time.localtime()
        current_time = time.strftime("%H:%M:%S", t)
        gen_str=    "S," +str(j)+ "," +str(txt)+ "," +str(Number_of_axle)+ "," +str(max_hgt)+ ",0.00,0.00,0.00," +str(first_two_axle_distance)+ "," +str(tandum)+ ", False, F," +str(my_string1)+ "," +str(td)+ "," +str(current_time)+ ",E"
        str_data(gen_str,j)
g=0        
def str_data(gen_str,j):
    global g
    td1 = date.today()
    td = td1.strftime("%d-%m-%Y")
    directory_path = 'C:/Users/Acer/Desktop/programs/pyserial/str_data'
    directory_name = td
    try:
        path = os.path.join(directory_path,directory_name)
        os.makedirs(path, exist_ok = True)
        new_file_path1 = os.path.join(path, str(j)+'.txt')
        new_file = open(new_file_path1,'a')
        new_file.writelines(str(gen_str))
        new_file.writelines('\n')
        new_file.close()
        thread1 = threading.Thread(target = send1, args=(gen_str,))
        #thread2= threading.Thread(target= accept1)
        thread1.start()
        #thread2.start()
    except OSError as error:
        pass

    
date_var = []
def send1(gen_str):
    global date_var, clientobject
    try:
        conn = True
        while conn:
            clientobject.sendall(gen_str.encode())
            msg = clientobject.recv(1024).decode('utf-8')
            if msg == 'disconnect':
                conn = False
            elif msg != '':
                try:
                    date_var.append(msg)
                    source = date_var[0]
                except IndexError:
                    pass
                try:
                    destination = date_var[1]
                    dump_data(source, destination)
                except IndexError:
                    pass
            elif msg == '':
                date_var.clear()
        clientobject.close()
        print(clientobject.fileno())
    except OSError:
        pass
    
allfiles=0
date1=[]
def dump_data(source, destination):
    global allfiles, date1
    base_path = 'C:/Users/Acer/Desktop/programs/HexData/'
    message = "Please Enter the path where you want to save your data"
    clientobject.send(message.encode())
    if destination!=0:
        dest_path = clientobject.recv(1024).decode()
    else:
        pass
    print("Before moving directories:")
    print(os.listdir(base_path))
    allfiles = os.listdir(base_path)
    date1 = [f for f in allfiles]
    date1.sort(key=lambda date1: datetime.strptime(date1, '%d-%m-%Y'))
    for i in range (len(date1)):
        sort_date.append(date1[i])
    s = sort_date.index(source)
    d = sort_date.index(destination)
    res = sort_date[s:d+1]
    for x in range (len(res)):
        dest = os.path.join(dest_path + res[x])
        source1 = os.path.join(base_path, res[x])
        shutil.move(source1, dest)
    print("After moving directories:")
    print(os.listdir(base_path))
    
while True:
    try:
        root.update()
        checkSerialPort()
        datacanvas.config(scrollregion = datacanvas.bbox("all"))
    except TclError:
        root.mainloop()
        break   
root.mainloop()